<a href="https://colab.research.google.com/github/PRASANNA-416/IT414-Mini-Project-/blob/main/MultiResUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
import os
import tensorflow as tf
from tqdm import tqdm
import cv2

input_dir = "/content/gdrive/MyDrive/ML/kvasir-seg/Kvasir-SEG/images"
target_dir = "/content/gdrive/MyDrive/ML/kvasir-seg/Kvasir-SEG/masks"

num_input_images = len(os.listdir(input_dir))
num_target_images = len(os.listdir(target_dir))

print("Number of input images:", num_input_images)
print("Number of target images:", num_target_images)

Number of input images: 1000
Number of target images: 1000


In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, add
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras import backend as K 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import Sequence
import os
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class CustomDataGenerator(Sequence):
    def __init__(self, input_dir, mask_dir, batch_size, target_size=(224, 224), shuffle=True):
        self.input_dir = input_dir
        self.mask_dir = mask_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle

        # Get list of image and mask file names
        self.filenames = os.listdir(self.input_dir)

        # Get number of batches per epoch
        self.steps_per_epoch = len(self.filenames) // self.batch_size

        # Shuffle filenames
        if self.shuffle:
            np.random.shuffle(self.filenames)

        # Set up data augmentation generator
        self.datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )

    def __len__(self):
        return self.steps_per_epoch

    def __getitem__(self, index):
        # Get batch indexes
        batch_filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]

        # Initialize batch arrays
        batch_input = np.zeros((self.batch_size, *self.target_size, 3))
        batch_mask = np.zeros((self.batch_size, *self.target_size, 1))

        # Load and augment batch images and masks
        for i, filename in enumerate(batch_filenames):
            # Load input image
            input_path = os.path.join(self.input_dir, filename)
            input_img = load_img(input_path, target_size=self.target_size)
            input_array = img_to_array(input_img)

            # Load target mask
            mask_path = os.path.join(self.mask_dir, filename)
            mask_img = load_img(mask_path, target_size=self.target_size, color_mode='grayscale')
            mask_array = img_to_array(mask_img)

            # Apply data augmentation to input and mask images
            input_array, mask_array = self.apply_data_augmentation(input_array, mask_array)

            # Normalize input image and mask
            input_array = input_array / 255.0
            mask_array = mask_array / 255.0

            # Add to batch arrays
            batch_input[i] = input_array
            batch_mask[i] = mask_array

        return batch_input, batch_mask

    def apply_data_augmentation(self, input_array, mask_array):
        # Combine input and mask arrays for data augmentation
        combined = np.concatenate([input_array, mask_array], axis=2)

        # Apply data augmentation
        augmented = self.datagen.random_transform(combined)

        # Split augmented array back into input and mask arrays
        aug_input = augmented[..., :3]
        aug_mask = augmented[..., 3:]

        return aug_input, aug_mask


In [4]:
image_dir = '/content/gdrive/MyDrive/ML/kvasir-seg/Kvasir-SEG/images'
mask_dir = '/content/gdrive/MyDrive/ML/kvasir-seg/Kvasir-SEG/masks'

# Define the batch size and target image size
batch_size = 10
target_size = (224, 224)

# Create the custom data generator
data_generator = CustomDataGenerator(image_dir, mask_dir, batch_size, target_size)

In [5]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, add
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras import backend as K 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [6]:
def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), activation='relu', name=None):
    '''
    2D Convolutional layers
    
    Arguments:
        x {keras layer} -- input layer 
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters
    
    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(1, 1)})
        activation {str} -- activation function (default: {'relu'})
        name {str} -- name of the layer (default: {None})
    
    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2D(filters, (num_row, num_col), strides=strides, padding=padding, use_bias=False)(x)
    x = BatchNormalization(axis=3, scale=False)(x)

    if(activation == None):
        return x

    x = Activation(activation, name=name)(x)

    return x


def trans_conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(2, 2), name=None):
    '''
    2D Transposed Convolutional layers
    
    Arguments:
        x {keras layer} -- input layer 
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters
    
    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(2, 2)})
        name {str} -- name of the layer (default: {None})
    
    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2DTranspose(filters, (num_row, num_col), strides=strides, padding=padding)(x)
    x = BatchNormalization(axis=3, scale=False)(x)
    
    return x


def MultiResBlock(U, inp, alpha = 1.67):
    '''
    MultiRes Block
    
    Arguments:
        U {int} -- Number of filters in a corrsponding UNet stage
        inp {keras layer} -- input layer 
    
    Returns:
        [keras layer] -- [output layer]
    '''

    W = alpha * U

    shortcut = inp

    shortcut = conv2d_bn(shortcut, int(W*0.167) + int(W*0.333) +
                         int(W*0.5), 1, 1, activation=None, padding='same')

    conv3x3 = conv2d_bn(inp, int(W*0.167), 3, 3,
                        activation='relu', padding='same')

    conv5x5 = conv2d_bn(conv3x3, int(W*0.333), 3, 3,
                        activation='relu', padding='same')

    conv7x7 = conv2d_bn(conv5x5, int(W*0.5), 3, 3,
                        activation='relu', padding='same')

    out = concatenate([conv3x3, conv5x5, conv7x7], axis=3)
    out = BatchNormalization(axis=3)(out)

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    return out


def ResPath(filters, length, inp):
    '''
    ResPath
    
    Arguments:
        filters {int} -- [description]
        length {int} -- length of ResPath
        inp {keras layer} -- input layer 
    
    Returns:
        [keras layer] -- [output layer]
    '''


    shortcut = inp
    shortcut = conv2d_bn(shortcut, filters, 1, 1,
                         activation=None, padding='same')

    out = conv2d_bn(inp, filters, 3, 3, activation='relu', padding='same')

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    for i in range(length-1):

        shortcut = out
        shortcut = conv2d_bn(shortcut, filters, 1, 1,
                             activation=None, padding='same')

        out = conv2d_bn(out, filters, 3, 3, activation='relu', padding='same')

        out = add([shortcut, out])
        out = Activation('relu')(out)
        out = BatchNormalization(axis=3)(out)

    return out


def MultiResUnet(height, width, n_channels):
    '''
    MultiResUNet
    
    Arguments:
        height {int} -- height of image 
        width {int} -- width of image 
        n_channels {int} -- number of channels in image
    
    Returns:
        [keras model] -- MultiResUNet model
    '''


    inputs = Input((height, width, n_channels))

    mresblock1 = MultiResBlock(32, inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(mresblock1)
    mresblock1 = ResPath(32, 4, mresblock1)

    mresblock2 = MultiResBlock(32*2, pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(mresblock2)
    mresblock2 = ResPath(32*2, 3, mresblock2)

    mresblock3 = MultiResBlock(32*4, pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(mresblock3)
    mresblock3 = ResPath(32*4, 2, mresblock3)

    mresblock4 = MultiResBlock(32*8, pool3)
    pool4 = MaxPooling2D(pool_size=(2, 2))(mresblock4)
    mresblock4 = ResPath(32*8, 1, mresblock4)

    mresblock5 = MultiResBlock(32*16, pool4)

    up6 = concatenate([Conv2DTranspose(
        32*8, (2, 2), strides=(2, 2), padding='same')(mresblock5), mresblock4], axis=3)
    mresblock6 = MultiResBlock(32*8, up6)

    up7 = concatenate([Conv2DTranspose(
        32*4, (2, 2), strides=(2, 2), padding='same')(mresblock6), mresblock3], axis=3)
    mresblock7 = MultiResBlock(32*4, up7)

    up8 = concatenate([Conv2DTranspose(
        32*2, (2, 2), strides=(2, 2), padding='same')(mresblock7), mresblock2], axis=3)
    mresblock8 = MultiResBlock(32*2, up8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(
        2, 2), padding='same')(mresblock8), mresblock1], axis=3)
    mresblock9 = MultiResBlock(32, up9)

    conv10 = conv2d_bn(mresblock9, 1, 1, 1, activation='sigmoid')
    
    model = Model(inputs=[inputs], outputs=[conv10])

    return model




In [7]:

from keras.metrics import MeanIoU

def dice_coefficient(y_true, y_pred, smooth=1.0):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true), axis=-1) + K.sum(K.square(y_pred), axis=-1) + smooth)


In [8]:
def trainStep(model, X_train, Y_train,epochs, batchSize):

    
    for epoch in range(epochs):
        print('Epoch : {}'.format(epoch+1))
        model.fit(x=X_train, y=Y_train, batch_size=batchSize, epochs=1, verbose=1)     


    return model 

In [9]:
model = MultiResUnet(height=224, width=224, n_channels=3)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 224, 224, 8)  216         ['input_1[0][0]']                
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 224, 224, 8)  24         ['conv2d_1[0][0]']               
 rmalization)                                                                                     
                                                                                              

In [10]:


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',dice_coefficient])



In [11]:
model.fit(data_generator, epochs=10,steps_per_epoch = 50)

Epoch 1/10
50/50 [==============================] - 427s 7s/step - loss: 0.6545 - accuracy: 0.6963 - dice_coefficient: 0.8431
Epoch 2/10
50/50 [==============================] - 179s 4s/step - loss: 0.6020 - accuracy: 0.7607 - dice_coefficient: 0.8540
Epoch 3/10
50/50 [==============================] - 103s 2s/step - loss: 0.5642 - accuracy: 0.8019 - dice_coefficient: 0.8629
Epoch 4/10
50/50 [==============================] - 48s 965ms/step - loss: 0.5358 - accuracy: 0.8148 - dice_coefficient: 0.8719
Epoch 5/10
50/50 [==============================] - 42s 835ms/step - loss: 0.5129 - accuracy: 0.8327 - dice_coefficient: 0.8782
Epoch 6/10
50/50 [==============================] - 33s 668ms/step - loss: 0.4964 - accuracy: 0.8457 - dice_coefficient: 0.8829
Epoch 7/10
50/50 [==============================] - 29s 574ms/step - loss: 0.4775 - accuracy: 0.8534 - dice_coefficient: 0.8892
Epoch 8/10
50/50 [==============================] - 23s 452ms/step - loss: 0.4567 - accuracy: 0.8713 - dice_co